In [88]:
from pathlib import Path
import importlib
import warnings
import time
import os
from datetime import datetime
import time
warnings.filterwarnings("ignore")  # Oculta todos los warnings

# Mis modulos
import process_data as dp
import transform_data as td

In [89]:
# import pyfiglet
# print(pyfiglet.figlet_format("Gestor de Vacaciones"))

banner = r"""
  ____           _                   _      
 / ___| ___  ___| |_ ___  _ __    __| | ___ 
| |  _ / _ \/ __| __/ _ \| '__|  / _` |/ _ \
| |_| |  __/\__ \ || (_) | |    | (_| |  __/
 \____|\___||___/\__\___/|_|     \__,_|\___|
                                            
__     __                   _                       
\ \   / /_ _  ___ __ _  ___(_) ___  _ __   ___  ___ 
 \ \ / / _` |/ __/ _` |/ __| |/ _ \| '_ \ / _ \/ __|
  \ V / (_| | (_| (_| | (__| | (_) | | | |  __/\__ \
   \_/ \__,_|\___\__,_|\___|_|\___/|_| |_|\___||___/

    💥 VISUALIZADOR Y EDITOR DE VACACIONES 💥
"""

print(banner, end='\n\n')
time.sleep(1)


  ____           _                   _      
 / ___| ___  ___| |_ ___  _ __    __| | ___ 
| |  _ / _ \/ __| __/ _ \| '__|  / _` |/ _ \
| |_| |  __/\__ \ || (_) | |    | (_| |  __/
 \____|\___||___/\__\___/|_|     \__,_|\___|

__     __                   _                       
\ \   / /_ _  ___ __ _  ___(_) ___  _ __   ___  ___ 
 \ \ / / _` |/ __/ _` |/ __| |/ _ \| '_ \ / _ \/ __|
  \ V / (_| | (_| (_| | (__| | (_) | | | |  __/\__ \
   \_/ \__,_|\___\__,_|\___|_|\___/|_| |_|\___||___/

    💥 VISUALIZADOR Y EDITOR DE VACACIONES 💥




In [ ]:
# Constantes
this_year = datetime.today().date().year

# Diccionarios
months = {
    1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril', 5: 'Mayo', 6: 'Junio',
    7: 'Julio', 8: 'Agosto', 9: 'Septiembre', 10: 'Octubre', 
    11: 'Noviembre', 12: 'Diciembre'
}
vacaciones = {
    "name": "Gestor de Vacaciones",
    "file_name_xlsx": "DATOS_VACACIONES_2020-2025.xlsx",
    "file_name_csv": "vacaciones_data.csv",
    "sheet_name": "DATOS",
    "relevant_columns": [
        'APELLIDOS',
        'NOMBRES',
        'DNI',
        'CARGO',
        'Fecha Ingreso',
        'Vacaciones 2020-2021',
        'Vacaciones 2021-2022',
        'Vacaciones 2022-2023',
        'Vacaciones 2023-2024',
        'Vacaciones 2024-2025',
    ],
    "new_columns": [
        "DIAS_ACUMULADOS",
        "VACACIONES_GOZADAS",
        "VACACION_GOZADA_ACTUAL", # 3 estados (No vacaciono, esta vacacionando, por vacacionar)
        "VACACIONES_PENDIENTES", # Dias de vacaciones acumuladas que tiene pendiente para gozar
    ]
}
VACACION_GOZADA_ACTUAL_ESTADOS = {
    0: f"Vacaciones no aplicables",
    1: f"Vacaciones del {this_year} gozadas",
    2: "Gozando vacaciones actualmente",
    3: f"Vacaciones del {this_year} por gozar",
}

# Listas

# Constantes
project_address = r"C:\Informacion\Otros\RRHH\Automatizacion\vacation_manager"

### Procesamiento de datos

In [91]:
importlib.reload(dp)

df = dp.main(project_address, vacaciones)

Schema({'APELLIDOS': String, 'NOMBRES': String, 'DNI': Int64, 'CARGO': String, 'Fecha Ingreso': String, 'Vacaciones 2020-2021': String, 'Vacaciones 2021-2022': String, 'Vacaciones 2022-2023': String, 'Vacaciones 2023-2024': String, 'Vacaciones 2024-2025': String})


### Transformacion de datos

In [92]:
importlib.reload(td)

df = td.main(df)

print(df)

shape: (65, 16)
┌───────────┬───────────┬──────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ APELLIDOS ┆ NOMBRES   ┆ DNI      ┆ CARGO     ┆ … ┆ VACACION_ ┆ ANIVERSAR ┆ VACACIONE ┆ VACACIONE │
│ ---       ┆ ---       ┆ ---      ┆ ---       ┆   ┆ GOZADA_AC ┆ IO        ┆ S_PROPORC ┆ S_PENDIEN │
│ str       ┆ str       ┆ i64      ┆ str       ┆   ┆ TUAL      ┆ ---       ┆ IONALES   ┆ TES       │
│           ┆           ┆          ┆           ┆   ┆ ---       ┆ datetime[ ┆ ---       ┆ ---       │
│           ┆           ┆          ┆           ┆   ┆ i32       ┆ μs]       ┆ f64       ┆ f64       │
╞═══════════╪═══════════╪══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ ALVARES   ┆ EDWIN     ┆ 70580389 ┆ CHOFER    ┆ … ┆ 3         ┆ 2024-03-0 ┆ 38.547945 ┆ 38.55     │
│ HUAMAN    ┆           ┆          ┆ VENDEDOR  ┆   ┆           ┆ 7         ┆           ┆           │
│           ┆           ┆          ┆           ┆   ┆           ┆ 00:00:00  